In [1]:
# pip install import_ipynb

In [5]:
import time
import os
import math
import argparse
from glob import glob
from collections import OrderedDict
import random
import warnings
from datetime import datetime
import joblib

import numpy as np
import pandas as pd
import import_ipynb

from sklearn.model_selection import KFold, StratifiedKFold

import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, LearningRateScheduler, TerminateOnNaN, LambdaCallback

import archs
from metrics import *
from scheduler import *


arch_names = archs.__dict__.keys()

def get_data():
    ds_train = tf.keras.preprocessing.image_dataset_from_directory(
        'data',
        labels="inferred",
        label_mode="int",
    #     class_names=None,
        color_mode="grayscale",
        batch_size=2,
        image_size=(28, 28),
        shuffle=True,
        seed=123,
        validation_split=0.2,
        subset="training"
    )

    ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
        'data',
        labels="inferred",
        label_mode="int",
    #     class_names=None,
        color_mode="grayscale",
        batch_size=2,
        image_size=(28, 28),
        shuffle=True,
        seed=123,
        validation_split=0.2,
        subset="validation"
    )
    
    train_label = np.concatenate([y for x, y in ds_train.map(lambda x, y: (tf.cast(x, tf.int16), y))], axis=0)
    train_data =  np.concatenate([x for x, y in ds_train.map(lambda x, y: (tf.cast(x, tf.int16), y))], axis=0)
    
    test_label = np.concatenate([y for x, y in ds_validation.map(lambda x, y: (tf.cast(x, tf.int16), y))], axis=0) 
    test_data =  np.concatenate([x for x, y in ds_validation.map(lambda x, y: (tf.cast(x, tf.int16), y))], axis=0)

    return (train_data, train_label), (test_data, test_label)

def parse_args():
    parser = argparse.ArgumentParser()

    parser.add_argument('--name', default='vgg8_cosface',
                        help='model name: (default: arch+timestamp)')
    parser.add_argument('--arch', '-a', metavar='ARCH', default='vgg8',
                        choices=arch_names,
                        help='model architecture: ' +
                            ' | '.join(arch_names) +
                            ' (default: vgg8)')
    parser.add_argument('--num-features', default=3, type=int,
                        help='dimention of embedded features')
    parser.add_argument('--scheduler', default='CosineAnnealing',
                        choices=['CosineAnnealing', 'None'],
                        help='scheduler: ' +
                            ' | '.join(['CosineAnnealing', 'None']) +
                            ' (default: CosineAnnealing)')
    parser.add_argument('--epochs', default=10, type=int, metavar='N',
                        help='number of total epochs to run')
    parser.add_argument('-b', '--batch-size', default=32, type=int,
                        metavar='N', help='mini-batch size (default: 128)')
    parser.add_argument('--optimizer', default='SGD',
                        choices=['Adam', 'SGD'],
                        help='loss: ' +
                            ' | '.join(['Adam', 'SGD']) +
                            ' (default: Adam)')
    parser.add_argument('--lr', '--learning-rate', default=1e-1, type=float,
                        metavar='LR', help='initial learning rate')
    parser.add_argument('--min-lr', default=1e-3, type=float,
                        help='minimum learning rate')
    parser.add_argument('--momentum', default=0.5, type=float)
    args = parser.parse_args("")

    return args


def main():
    args = parse_args()

    # add model name to args
    args.name = 'faces_%s_%dd' %(args.arch, args.num_features)

    os.makedirs('models/%s' %args.name, exist_ok=True)

    print('Config -----')
    for arg in vars(args):
        print('%s: %s' %(arg, getattr(args, arg)))
    print('------------')

    joblib.dump(args, 'models/%s/args.pkl' %args.name)

    with open('models/%s/args.txt' %args.name, 'w') as f:
        for arg in vars(args):
            print('%s: %s' %(arg, getattr(args, arg)), file=f)

#     (X, y), (X_test, y_test) = mnist.load_data()
    (X, y), (X_test, y_test) = get_data()

    X = X[:, :, :, np.newaxis].astype('float32') / 255
    X_test = X_test[:, :, :, np.newaxis].astype('float32') / 255

    y = tensorflow.keras.utils.to_categorical(y, 10)
    y_test = tensorflow.keras.utils.to_categorical(y_test, 10)

    if args.optimizer == 'SGD':
        optimizer = SGD(lr=args.lr, momentum=args.momentum)
    elif args.optimizer == 'Adam':
        optimizer = Adam(lr=args.lr)

    model = archs.__dict__[args.arch](args)
    model.compile(loss='categorical_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
    model.summary()

    callbacks = [
        ModelCheckpoint(os.path.join('models', args.name, 'model.hdf5'),
            verbose=1, save_best_only=True),
        CSVLogger(os.path.join('models', args.name, 'log.csv')),
        TerminateOnNaN()]

    if args.scheduler == 'CosineAnnealing':
        callbacks.append(CosineAnnealingScheduler(T_max=args.epochs, eta_max=args.lr, eta_min=args.min_lr, verbose=1))

    if 'face' in args.arch:
        # callbacks.append(LambdaCallback(on_batch_end=lambda batch, logs: print('W has nan value!!') if np.sum(np.isnan(model.layers[-4].get_weights()[0])) > 0 else 0))
        model.fit([X, y], y, validation_data=([X_test, y_test], y_test),
            batch_size=args.batch_size,
            epochs=args.epochs,
            callbacks=callbacks,
            verbose=1)
    else:
        model.fit(X, y, validation_data=(X_test, y_test),
            batch_size=args.batch_size,
            epochs=args.epochs,
            callbacks=callbacks,
            verbose=1)

    model.load_weights(os.path.join('models/%s/model.hdf5' %args.name))
    if 'face' in args.arch:
        score = model.evaluate([X_test, y_test], y_test, verbose=1)
    else:
        score = model.evaluate(X_test, y_test, verbose=1)

    print("Test loss:", score[0])
    print("Test accuracy:", score[1])


if __name__ == '__main__':
    main()


Config -----
name: faces_vgg8_3d
arch: vgg8
num_features: 3
scheduler: CosineAnnealing
epochs: 10
batch_size: 32
optimizer: SGD
lr: 0.1
min_lr: 0.001
momentum: 0.5
------------
Found 902 files belonging to 5 classes.
Using 722 files for training.
Found 902 files belonging to 5 classes.
Using 180 files for validation.
Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
batch_normalization_24 (Batc (None, 28, 28, 16)        64        
_________________________________________________________________
activation_18 (Activation)   (None, 28, 28, 16)        0         
_________________________________________________________________

6/6 [==============================] - 0s 10ms/step - loss: 1.6777 - accuracy: 0.3333
Test loss: 1.6776689291000366
Test accuracy: 0.3333333432674408


In [ ]:
# !python train.py --arch vgg8_cosface